# TP FINAL Debandi

## Carga de datos y preparación del dataset

En este trabajo utilizo los datos de Covid-19 junto con un dataset complementario de información sobre cuando cada persona es migrante o no.

Este notebook levanta tres archivos: 
casosCovid: es un archivo de casos de marzo 2020 hasta mayo 2021. Este archivo fue bajado desde el sitio de datos abiertos del Ministerio de Salud de Argentina
http://datos.salud.gob.ar/dataset?groups=covid-19

migrantes2020 y migrantes2021: ambos archivos corresponden a dos pedidos de acceso a la información publica realizados en 2021. Este archivo contempla el ID-caso y algunos campos para dar cuenta de la nacionalidad y el país de nacimiento de cada persona.

La cantidad de registros del archivo de casos es dee 15.071.716 
Dado el volumen y el tiempo que llevaba este procesamiento de manera local, para este trabajo se realizó una muesta del 10%.


Esta notebook levanta los archivos de casos y toma una muestra del 10%., Hace un join con los datos de migrantes y se seleccionan las variables de interés.
Luego se guardan los datos en formato parquet. En la notebook de análisis se toman directamente estos datos.

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("pyspark-intro")
    .config("spark.driver.memory", "512m")
    .config("spark.driver.cores", "1")
    .config("spark.executor.memory", "512m")
    .config("spark.executor.cores", "1")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)

# Levanto los datos y los exploro

Levanto el dataset de casos con los tipos de cada columna para poder trabajarlos

In [3]:
#especifico el esquema para poder definir los tipos
from pyspark.sql.types import *

schema = (StructType()
          .add("id_evento_caso", IntegerType())
          .add("sexo", StringType())
          .add("edad", IntegerType())
          .add("edad_años_meses", IntegerType())
          .add("residencia_pais_nombre", StringType())
          .add("residencia_provincia_nombre", StringType())
          .add("residencia_departamento_nombre", StringType())
          .add("carga_provincia_nombre", StringType())
          .add("fecha_inicio_sintomas", DateType())

          .add("fecha_apertura", DateType())
          .add("sepi_apertura", IntegerType())
          .add("fecha_internacion", DateType())
          .add("cuidado_intensivo", StringType())
          .add("fecha_cui_intensivo", DateType())
          .add("fallecido", StringType())
          .add("fecha_fallecimiento", DateType())
          
          .add("asistencia_respiratoria_mecanica", DateType())
          .add("carga_provincia_id", IntegerType())
          .add("origen_financiamiento", StringType())
          .add("clasificacion", StringType())
          .add("clasificacion_resumen", StringType())
          .add("residencia_provincia_id", DateType())
          .add("fecha_diagnostico", DateType())
          .add("residencia_departamento_id", IntegerType())
          .add("ultima_actualizacion", IntegerType())

         )

In [4]:
#levanto los casos
dfcasosCompleto = spark.read.csv("/dataset/covid/Covid19Casos.csv", header=True)

#dfcasosCompleto = spark.read.csv("/dataset/covid/Covid19Casos.csv", 
#                        schema=schema, header=True)



In [5]:
##l .show muestra todo null. Luego la información está bien.
dfcasosCompleto.show(5)


In [6]:
dfcasosCompleto.count()

Hay un total de 15.071.716 casos.

Cambio los tipos de las columnas. NO pude importarlo con el schema. Me trajo multiples errores

In [7]:
#Randomly sample 10% of the data without replacement
dfcasos = dfcasosCompleto.sample(False, 0.10, seed=7)

In [8]:
dfcasos.count()

1507618

Cantidad de casos: 1.507.101

In [9]:
dfcasos.printSchema()

root
 |-- id_evento_caso: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- edad_años_meses: integer (nullable = true)
 |-- residencia_pais_nombre: string (nullable = true)
 |-- residencia_provincia_nombre: string (nullable = true)
 |-- residencia_departamento_nombre: string (nullable = true)
 |-- carga_provincia_nombre: string (nullable = true)
 |-- fecha_inicio_sintomas: date (nullable = true)
 |-- fecha_apertura: date (nullable = true)
 |-- sepi_apertura: integer (nullable = true)
 |-- fecha_internacion: date (nullable = true)
 |-- cuidado_intensivo: string (nullable = true)
 |-- fecha_cui_intensivo: date (nullable = true)
 |-- fallecido: string (nullable = true)
 |-- fecha_fallecimiento: date (nullable = true)
 |-- asistencia_respiratoria_mecanica: date (nullable = true)
 |-- carga_provincia_id: integer (nullable = true)
 |-- origen_financiamiento: string (nullable = true)
 |-- clasificacion: string (nullable = true)
 |-- clasif

In [10]:
dfcasos.write.parquet("/dataset/parquet/covidcasos.parquet") 

# levanto el dataset de migrantes

Voy a levantar ahora los dos archivos de migrantes, unirlos. Estos archivos corresponden a dos pedidos de información distintos.

In [11]:
schema_mig = (StructType()
          .add("id_caso", IntegerType())
          .add("nacionalidad", StringType())
          .add("pais_nacimiento", StringType())
          .add("tipo_doc", StringType())
          .add("nro_doc", IntegerType())
          .add("doc_argentino", IntegerType())
          .add("base", IntegerType())
         )

In [12]:
#dfmig1 = spark.read.option("delimiter", ";").csv("/dataset/covid/migrantes2020.csv", header=True)


In [13]:
dfmig1 = (spark.read.option("delimiter", ";").csv("/dataset/covid/migrantes2020.csv", 
                         schema=schema_mig, header=True, 
                         ignoreLeadingWhiteSpace=True, 
                         ignoreTrailingWhiteSpace=True))


In [14]:
#dfmig1.show()

In [15]:
dfmig2 = (spark.read.option("delimiter", ";").csv("/dataset/covid/migrantes2021.csv", 
                         schema=schema_mig, header=True, 
                         ignoreLeadingWhiteSpace=True, 
                         ignoreTrailingWhiteSpace=True))

In [16]:
dfmig2.printSchema()

root
 |-- id_caso: integer (nullable = true)
 |-- nacionalidad: string (nullable = true)
 |-- pais_nacimiento: string (nullable = true)
 |-- tipo_doc: string (nullable = true)
 |-- nro_doc: integer (nullable = true)
 |-- doc_argentino: integer (nullable = true)
 |-- base: integer (nullable = true)



In [17]:
dfmig= dfmig1.union(dfmig2)

# To remove the duplicates:

#result = result.dropDuplicates()


In [18]:
dfmig.printSchema()

root
 |-- id_caso: integer (nullable = true)
 |-- nacionalidad: string (nullable = true)
 |-- pais_nacimiento: string (nullable = true)
 |-- tipo_doc: string (nullable = true)
 |-- nro_doc: integer (nullable = true)
 |-- doc_argentino: integer (nullable = true)
 |-- base: integer (nullable = true)



In [19]:
dfmig.count()

11437754

In [22]:
##extraigo las columnas que me interesan para achicar el dataset
dfmig.createOrReplaceTempView("dfmig")

dfTemp2= spark.sql(""" SELECT id_caso,pais_nacimiento
    FROM dfmig 
    """)

In [23]:
#dfmig.count()
#vuelvo a chequear la distribución de fallecido
#dfmig.groupBy('pais_nacimiento').count().orderBy('count').show()

In [25]:
dfTemp2.write.parquet("/dataset/parquet/covidmigrantes2.parquet") 

In [28]:
test1=spark.read.parquet("/dataset/parquet/covidcasos.parquet") 

In [29]:
test1.printSchema()

root
 |-- id_evento_caso: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- edad_años_meses: integer (nullable = true)
 |-- residencia_pais_nombre: string (nullable = true)
 |-- residencia_provincia_nombre: string (nullable = true)
 |-- residencia_departamento_nombre: string (nullable = true)
 |-- carga_provincia_nombre: string (nullable = true)
 |-- fecha_inicio_sintomas: date (nullable = true)
 |-- fecha_apertura: date (nullable = true)
 |-- sepi_apertura: integer (nullable = true)
 |-- fecha_internacion: date (nullable = true)
 |-- cuidado_intensivo: string (nullable = true)
 |-- fecha_cui_intensivo: date (nullable = true)
 |-- fallecido: string (nullable = true)
 |-- fecha_fallecimiento: date (nullable = true)
 |-- asistencia_respiratoria_mecanica: date (nullable = true)
 |-- carga_provincia_id: integer (nullable = true)
 |-- origen_financiamiento: string (nullable = true)
 |-- clasificacion: string (nullable = true)
 |-- clasif

In [30]:
test1.count()

1507618

In [32]:
test2=spark.read.parquet("/dataset/parquet/covidmigrantes2.parquet") 

In [33]:
test2.printSchema()

root
 |-- id_caso: integer (nullable = true)
 |-- pais_nacimiento: string (nullable = true)



In [34]:
test2.count()

11437754